In [290]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.snowball import SnowballStemmer #porterstemmer?
from scipy.sparse.linalg import svds
from scipy.stats import percentileofscore
from surprise import SVD
from surprise import Dataset
from surprise import Reader

In [291]:
def getpredictions( predictions, userId ):
    user_pred = []
    for uid, iid, true_r, est, _ in predictions:
        if uid == userId :
            user_pred.append( (iid, est) )
    df_userp = pd.DataFrame( user_pred, columns = [ 'movieId', 'rating_pred'] )
    df_userp['movieId'] = df_userp['movieId'].astype(int)
    return( df_userp )

In [292]:
df_md_small = pd.read_csv('../movielens_small/metadata_clean.csv')
df_small = pd.read_csv('../movielens_small/ratings_clean.csv')

In [293]:
reader = Reader()
rating_data = Dataset.load_from_df(df_small[['userId', 'movieId', 'rating']], reader)
#rating_data.split(n_folds=5)

In [294]:
algo = SVD( biased = True, n_factors = 100, n_epochs = 25 )
trainset = rating_data.build_full_trainset()
algo.fit(trainset)

In [295]:
# predict ratings for all pairs (u, i) that are not in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)

In [296]:
df_md_small['genres'] = df_md_small['genres'].apply(ast.literal_eval)
df_md_small['cast'] = df_md_small['cast'].apply(ast.literal_eval)
df_md_small['keywords'] = df_md_small['keywords'].apply(ast.literal_eval)

In [297]:
df_md_small['director_'] = ( df_md_small['director'].
                            astype(str).
                            apply( lambda x: x.lower().replace( " ", "") ).
                            apply( lambda x: [x, x]) ) # more weights

In [298]:
df_md_small['cast_'] = ( df_md_small['cast'].
                        apply( lambda x: [i.lower().replace( " ", "") for i in x ]) )

In [299]:
df_md_small['keywords_'] = ( df_md_small['keywords'].
                            apply( lambda x: [SnowballStemmer( 'english' ).stem( i ) for i in x]).
                            apply( lambda x: [i.lower().replace( " ", "") for i in x ]) )

In [300]:
df_md_small['content'] = ( df_md_small['cast_'] +
                          df_md_small['director_'] +
                          df_md_small['keywords_'] +
                          df_md_small['genres'] ).apply( lambda x: ' '.join( x ) ) # combine all content features

In [301]:
content_matrix = CountVectorizer(
    analyzer = 'word',
    ngram_range = (1, 2),
    min_df = 0,
    stop_words = 'english').fit_transform(df_md_small['content'])
cosine_sim = cosine_similarity(content_matrix, content_matrix)

In [302]:
user_id = 50
movie_title = 'Terminator 2: Judgment Day'
df_md_small.loc[ df_md_small['title'] == movie_title, ['title', 'director', 'genres', 'cast'] ]

,title,director,genres,cast
522,Terminator 2: Judgment Day,James Cameron,"[Action, Thriller, Science Fiction]","[Arnold Schwarzenegger, Linda Hamilton, Robert..."


In [303]:
df_user_rating = ( df_small.loc[ df_small['userId'] == user_id ].
               merge(df_md_small, on = 'movieId').
               reset_index(drop = True) )

In [304]:
# user's top rated movies
df_user_rating[ ['movieId', 'title', 'rating', 'genres'] ].sort_values('rating', ascending = False ).head(10)

,movieId,title,rating,genres
41,589,Terminator 2: Judgment Day,5.0,"[Action, Thriller, Science Fiction]"
0,10,GoldenEye,4.0,"[Adventure, Action, Thriller]"
15,292,Outbreak,4.0,"[Action, Drama, Science Fiction, Thriller]"
44,780,Independence Day,4.0,"[Action, Adventure, Science Fiction]"
37,527,Schindler's List,4.0,"[Drama, History, War]"
35,480,Jurassic Park,4.0,"[Adventure, Science Fiction]"
31,440,Dave,4.0,[Comedy]
26,368,Maverick,4.0,"[Action, Adventure, Comedy, Drama, Western]"
25,367,The Mask,4.0,"[Romance, Comedy, Crime, Fantasy]"
24,357,Four Weddings and a Funeral,4.0,"[Comedy, Drama, Romance]"


In [305]:
df_userp = getpredictions( predictions, user_id )
df_userp.shape

(8979, 2)

In [306]:
# top recommendations using CF
df_userp.sort_values( 'rating_pred', ascending = False ).merge( df_md_small[['movieId', 'title']], on = 'movieId').head(10)

,movieId,rating_pred,title
0,318,4.372454,The Shawshank Redemption
1,2329,4.276229,American History X
2,116797,4.274088,The Imitation Game
3,1221,4.230524,The Godfather: Part II
4,969,4.228466,The African Queen
5,1228,4.211642,Raging Bull
6,898,4.204301,The Philadelphia Story
7,899,4.203286,Singin' in the Rain
8,76093,4.201527,How to Train Your Dragon
9,2542,4.197234,"Lock, Stock and Two Smoking Barrels"


In [307]:
idx = np.asscalar( df_md_small.loc[ df_md_small['title'] == movie_title ].index.values[0] )
df_cbp = df_md_small[['movieId', 'title', 'genres']].reset_index( drop = True )
df_cbp[ 'sim' ] = cosine_sim[ idx ].flatten().tolist()
df_cbp.shape

(9082, 4)

In [308]:
# top recommendations using content-based filtering (CBF)
df_cbp.sort_values( 'sim', ascending = False ).head(11)

,movieId,title,genres,sim
522,589,Terminator 2: Judgment Day,"[Action, Thriller, Science Fiction]",1.000000
990,1240,The Terminator,"[Action, Thriller, Science Fiction]",0.367653
7404,73321,The Book of Eli,"[Action, Thriller, Science Fiction]",0.284029
3671,4691,Def-Con 4,"[Horror, Science Fiction, Thriller, Action]",0.243757
6821,56174,I Am Legend,"[Drama, Horror, Action, Thriller, Science Fict...",0.240048
953,1200,Aliens,"[Horror, Action, Thriller, Science Fiction]",0.235294
2024,2531,Battle for the Planet of the Apes,"[Action, Science Fiction]",0.234023
901,1127,The Abyss,"[Adventure, Action, Thriller, Science Fiction]",0.230812
2430,3032,The Omega Man,"[Action, Science Fiction, Thriller]",0.230205
2951,3702,Mad Max,"[Adventure, Action, Thriller, Science Fiction]",0.222566


In [309]:
df_c = df_userp.merge( df_cbp, on = 'movieId' )
x = df_c['rating_pred'].values
y = df_c['sim'].values
x_p = np.array( [percentileofscore(x, i, 'mean') for i in x] )
y_p = np.array( [percentileofscore(y, i, 'mean') for i in y] )
df_c['rating_p'] = x_p.tolist()
df_c['sim_p'] = y_p.tolist()

In [310]:
weight_cb = 0.5
score = x_p * ( 1 - weight_cb ) + y_p * weight_cb
df_c['score'] = score.tolist()

In [311]:
# top recommendations from hybrid recommendation engine
df_c.sort_values( 'score', ascending = False ).iloc[0:10][ ['title'] ].merge( df_md_small[['title', 'director', 'genres', 'cast']], on = 'title').reset_index( drop = True )

,title,director,genres,cast
0,Mad Max 2: The Road Warrior,George Miller,"[Adventure, Action, Thriller, Science Fiction]","[Mel Gibson, Bruce Spence, Michael Preston, Ma..."
1,Mad Max: Fury Road,George Miller,"[Action, Adventure, Science Fiction, Thriller]","[Tom Hardy, Charlize Theron, Nicholas Hoult, H..."
2,The Terminator,James Cameron,"[Action, Thriller, Science Fiction]","[Arnold Schwarzenegger, Michael Biehn, Linda H..."
3,Equilibrium,Kurt Wimmer,"[Action, Science Fiction, Thriller]","[Christian Bale, Taye Diggs, Emily Watson, Ang..."
4,Star Trek: First Contact,Jonathan Frakes,"[Science Fiction, Action, Adventure, Thriller]","[Patrick Stewart, Jonathan Frakes, Brent Spine..."
5,Minority Report,Steven Spielberg,"[Action, Thriller, Science Fiction, Mystery]","[Tom Cruise, Colin Farrell, Samantha Morton, M..."
6,28 Days Later,Danny Boyle,"[Horror, Thriller, Science Fiction]","[Cillian Murphy, Naomie Harris, Noah Huntley, ..."
7,The Matrix,Lana Wachowski,"[Action, Science Fiction]","[Keanu Reeves, Laurence Fishburne, Carrie-Anne..."
8,Ghost in the Shell,Mamoru Oshii,"[Action, Animation, Science Fiction]","[Atsuko Tanaka, Akio Ohtsuka, Iemasa Kayumi, K..."
9,Delicatessen,Jean-Pierre Jeunet,"[Comedy, Science Fiction, Fantasy]","[Dominique Pinon, Marie-Laure Dougnac, Jean-Cl..."


In [312]:
# explanation for top recommendations from hybrid recommendation engine
df_c.sort_values( 'score', ascending = False ).iloc[0:10][['title', 'rating_pred', 'sim', 'rating_p', 'sim_p', 'score']].reset_index( drop = True )

,title,rating_pred,sim,rating_p,sim_p,score
0,Mad Max 2: The Road Warrior,3.940622,0.197446,98.557746,99.849649,99.203698
1,Mad Max: Fury Road,3.896748,0.196078,97.844972,99.810669,98.827820
2,The Terminator,3.820030,0.367653,96.274641,99.994431,98.134536
3,Equilibrium,3.843986,0.131212,96.820359,98.919702,97.870030
4,Star Trek: First Contact,3.878909,0.115406,97.577681,98.140105,97.858893
5,Minority Report,3.790385,0.163401,95.573004,99.509968,97.541486
6,28 Days Later,3.883106,0.107572,97.689052,97.354939,97.521996
7,The Matrix,4.003492,0.094407,99.125738,95.912685,97.519211
8,Ghost in the Shell,3.809873,0.125245,96.029625,98.624568,97.327097
9,Delicatessen,3.810064,0.121578,96.040762,98.496492,97.268627
